In [1]:
from pathlib import Path
import ROOT
%jsroot on

Welcome to JupyROOT 6.24/06


A note on the EIC tutorials and/or ROOT class reference documentation: these use C++ syntax, while this notebook is in Python. Generally the names of the methods are the same, so in most cases it should be straightforward to "translate" between the two.

One of the biggest differences is how objects are created, e.g. compare the C++ syntax used in the EIC tutorial, with the Python equivalent:

```c++
// C++
TVector3 electron_mc(0., 0., 0.);
electron_mc.SetXYZ(123., 234., 345.);
```

```python
# Python
electron_mc = TVector3(0., 0., 0.)
electron_mc.SetXYZ(123., 234., 345.)
```

In [2]:
data_path = Path("./data")
assert data_path.is_dir()

A `TChain` allows us to add data from multiple file, and then refer to that object as if it was a normal `TTree`:

- https://root.cern.ch/doc/master/classTTree.html
- https://root.cern.ch/doc/master/classTChain.html

In [3]:
chain = ROOT.TChain("events")
for particle in ["e-", "mu-", "pi-"]:
    file_name = f"{particle}_8GeV_eta0.0.0000.eicrecon.tree.edm4eic.root"
    file_path = data_path / file_name
    chain.Add(str(file_path))

In [4]:
tree = chain
tree

Warning in <TClass::Init>: no dictionary for class podio::version::Version is available
Warning in <TClass::Init>: no dictionary for class edm4eic::ClusterData is available
Warning in <TClass::Init>: no dictionary for class edm4eic::Cov3f is available
Warning in <TClass::Init>: no dictionary for class edm4eic::Cov2f is available
Warning in <TClass::Init>: no dictionary for class edm4eic::CalorimeterHitData is available
Warning in <TClass::Init>: no dictionary for class edm4eic::TrackerHitData is available
Warning in <TClass::Init>: no dictionary for class edm4eic::CovDiag3f is available
Warning in <TClass::Init>: no dictionary for class edm4eic::TrackParametersData is available
Warning in <TClass::Init>: no dictionary for class edm4hep::Vector2f is available
Warning in <TClass::Init>: no dictionary for class edm4eic::TrackSegmentData is available
Warning in <TClass::Init>: no dictionary for class edm4eic::TrackPoint is available
Warning in <TClass::Init>: no dictionary for class edm4ei

In [5]:
tree.GetEntries()

95641

In [6]:
tree.Print("EcalBarrelScFiRecHits*")

******************************************************************************
*Chain   :events    : data/e-_8GeV_eta0.0.0000.eicrecon.tree.edm4eic.root    *
******************************************************************************
******************************************************************************
*Tree    :events    : Events tree                                            *
*Entries :    20000 : Total =       842857279 bytes  File  Size =  247835195 *
*        :          : Tree compression factor =   3.40                       *
******************************************************************************
*Br    0 :EcalBarrelScFiRecHits : Int_t EcalBarrelScFiRecHits_               *
*Entries :    20000 : Total  Size=     190671 bytes  File Size  =      55771 *
*Baskets :        8 : Basket Size=      32000 bytes  Compression=   2.88     *
*............................................................................*
*Br    1 :EcalBarrelScFiRecHits.cellID :            

`TTreeReader` is a class that helps iterating over a tree. First, you create a `TTreeReader` object, passing our `tree` as an argument:

In [7]:
reader = ROOT.TTreeReader(tree)

Then, you define references to the data in the `TTree`'s branches using `TTreeReaderArray` objects of the appropriate type.

In the following example, we create a `TTreeReaderArray` object for the PDG ID of the generated particles:

- The data type, in this case`int`, is passed inside square brackets before the other arguments
- The first argument is the `TTreeReader` object where the data is located
- Finally, the second argument is the name of the branch, as it appears in e.g. `TBrowser`, `TTree.Print()`, etc



In [8]:
PDG_mc = ROOT.TTreeReaderArray[int](reader, "GeneratedParticles.PDG")

By itself, the `TTreeReaderArray` object doesn't do much. However, calling the `Next()` method of the `TTreeReader` object will automatically populate each `TTreeReaderArray` object associated with it with the data from its connected branch for a single entry in the tree.

In other words, we only need to _create_ the `TTreeReaderArray` objects once, and then we _use_ them $N$ times, where $N$ is the number of entries in the tree.

More information:

- https://root.cern.ch/doc/master/classTTreeReader.html
- https://root.cern.ch/doc/master/classTTreeReaderArray.html

To read data for a `TTreeReaderArray` object, we use the fact that it is a _Python iterable_, a generalized sequence-like object. This means that we can:

- Iterate over it using `for item in PDG_mc: ...`
- Convert it a Python `list` object: `PDG_as_list = list(PDG_mc)`
- Using any of the Python built-in functions that support iterables, e.g. `len()` to get its length, etc

One useful Python function that we'll use is `zip()`, which allows to iterate "side-by-side" by converting $M$ iterables of length $N$ into $N$ tuples with $M$ elements each:

In [9]:
# we have M = 2 iterables (in this case lists) with N = 4 items each
names = ["e-", "mu-", "proton", "neutron"]
pdg_ids = [-11, -13, 2212, 2112]

# zip() converts them into N = 4 tuples with M = 2 elements each,
# that we can "unpack" (assign their elements to separate variables) directly in the loop
for name, pdg_id in zip(names, pdg_ids):
    print(name, pdg_id)

e- -11
mu- -13
proton 2212
neutron 2112


More information on Python iterables and `zip()`:

- https://realpython.com/python-zip-function/
- https://realpython.com/python-iterators-iterables/#getting-to-know-python-iterables

We can define all the `TTreeReaderArray`s we need for our analysis in one place:

In [10]:
PDG_mc = ROOT.TTreeReaderArray[int](reader, "GeneratedParticles.PDG")
type_mc = ROOT.TTreeReaderArray[int](reader, "GeneratedParticles.type")


scfi_rec_energy = ROOT.TTreeReaderArray[float](reader, "EcalBarrelScFiRecHits.energy")
scfi_rec_layer = ROOT.TTreeReaderArray[int](reader, "EcalBarrelScFiRecHits.layer")

We do the same for the histograms:

- Reference for `TH1` classes: https://root.cern/doc/master/classTH1.html

In [11]:
h_scfi_rec_energy_e = ROOT.TH1F("scfi_rec_energy_e", "ScFi reconstructed energy for e", 100, 0, 2)
h_scfi_rec_energy_mu = ROOT.TH1F("scfi_rec_energy_mu", "ScFi reconstructed energy for mu", 100, 0, 2)
h_scfi_rec_energy_pi = ROOT.TH1F("scfi_rec_energy_pi", "ScFi reconstructed energy for pi", 100, 0, 2)

We're now ready to run the analysis by iterating over the entries in the tree. This will have the form of a nested loop with at least two levels:

- The _event loop_, where each item represents a single event
- Within a single event, one or more _inner loops_ over various arrays of interest, e.g. reconstructed particles and ScFi reconstructed hits in our case

Note that the size of the inner loops in general differs from one event to the other: e.g. Event #123 might have 83 reconstructed ScFi hits, while Event #456 could have 72 hits, etc.

In [12]:
map_PDG_to_name = {
    -11: "e-",
    11: "e+",
    -13: "mu-",
    13: "mu+",
    -211: "pi-",
    211: "pi+",
}

evt_idx = 0
while reader.Next():  # iterate over the tree reader, moving to the next event if available, otherwise exit loop
    evt_idx += 1

    # we use the generated (MC) information to determine which primary particle we have for this event
    particle = None
    for gen_particle_PDG, gen_particle_type in zip(PDG_mc, type_mc):
        # 1 for primary, 2 for secondary
        if gen_particle_type == 1:
            assert gen_particle_PDG in map_PDG_to_name
            particle = map_PDG_to_name[gen_particle_PDG]
    assert particle is not None
    # then, we loop over the reconstructed ScFi hits by iterating on the `TTreeReaderArray` objects we created
    # for the `EcalBarrelScFiRecHits` branches
    energy_first_layers = 0
    for energy, layer in zip(scfi_rec_energy, scfi_rec_layer):
        if layer < 3:  # only consider the first 3 layers
            energy_first_layers += energy

    # finally, we fill the appropriate histogram depending on which primary particle we have in this event
    if particle.startswith("e"):
        h_scfi_rec_energy_e.Fill(energy_first_layers)
    elif particle.startswith("mu"):
        h_scfi_rec_energy_mu.Fill(energy_first_layers)
    elif particle.startswith("pi"):
        h_scfi_rec_energy_pi.Fill(energy_first_layers)
        
    # end of the event loop code block; the iteration will start again for the next event (if available)
        

In [13]:
c = ROOT.TCanvas("rec_energy", "ScFi reconstructed energy for first 3 layers")

- `TCanvas` class reference: https://root.cern.ch/doc/master/classTCanvas.html
- `TColor` class reference: https://root.cern.ch/doc/master/classTColor.html
  - "The color wheel" section

In [14]:
for h, color in [
    (h_scfi_rec_energy_e, ROOT.kRed),
    (h_scfi_rec_energy_mu, ROOT.kBlue),
    (h_scfi_rec_energy_pi, ROOT.kGreen - 2),
]:
    # TODO fix title, axis limits, legend, ...
    print(h.GetEntries())
    h.SetLineColor(color)
    h.Draw("same")

20000.0
50000.0
25641.0


In [15]:
c.Draw()